In [1]:
import os
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil
import sys
import pandas as pd
import geopandas as gpd
import contextily as cx
import flopy
from modflowapi import ModflowApi
from modflowapi.extensions import ApiSimulation
import itertools
from bmi.wrapper import BMIWrapper

import pyswmm
from pyswmm import Simulation, Nodes
from pyswmm import Output
import dask

c:\Users\bbayrakt\AppData\Local\miniforge3\envs\liss\lib\site-packages\bmi\__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
c:\Users\bbayrakt\AppData\Local\miniforge3\envs\liss\lib\site-packages\pkg_resources\__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
c:\Users\bbayrakt\AppData\Local\miniforge3\envs\liss\lib\site-packages\pkg_resources\__init__.py:2832: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap.responses')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/e

In [2]:
sys.path.append("../common")
from liss_settings import \
    libmf6, \
    get_dflow_grid_name, get_dflow_dtuser, \
    get_modflow_coupling_tag, get_modflow_grid_name, \
    silent, verbosity, \
    print_path, print_value

import swmm_mf_connect_1

# Load Names

dflow

In [3]:
control_path = pl.Path("../dflow-fm/coarse/tides_2018_atm/base/FlowFM.mdu") # change this if using a different D-Flow FM control file
dflow_grid_name = get_dflow_grid_name(control_path)
print(dflow_grid_name)
dflowfm_dtuser = get_dflow_dtuser(control_path)
print(f'{dflowfm_dtuser} seconds')

LIS_GPT_PJ_cells3_net
300.0 seconds


modflow

In [4]:
mf_grid_name = get_modflow_grid_name()
mf_grid_name

'PJmf6'

swmm

In [5]:
swmm_path = pl.Path("../swmm/PJ/SWMM_Latest/SWMM_PJ_Tidalgate_Sept-Oct2018_1min.inp").resolve()
print(swmm_path, swmm_path.is_file())

fpth = swmm_path.with_suffix(".out")
if fpth.is_file():
    print(f"removing: '{fpth}'")
    fpth.unlink()
fpth = swmm_path.with_suffix(".rpt")
if fpth.is_file():
    print(f"removing: '{fpth}'")
    fpth.unlink()

D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\swmm\PJ\SWMM_Latest\SWMM_PJ_Tidalgate_Sept-Oct2018_1min.inp True
removing: 'D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\swmm\PJ\SWMM_Latest\SWMM_PJ_Tidalgate_Sept-Oct2018_1min.out'
removing: 'D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\swmm\PJ\SWMM_Latest\SWMM_PJ_Tidalgate_Sept-Oct2018_1min.rpt'


#  Unit Conversions

In [6]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((m2ft**3) * 86400.)

# MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [7]:
mf_couple_freq_hours = 1 # Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
dflow_per_mf = int(mf_couple_freq / dflowfm_dtuser)
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to D-FLOW FM every {dflow_per_mf} output time step ({dflowfm_dtuser} sec.)") 

mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

mf_couple_nstp = int(86400.0 / (dflow_per_mf * dflowfm_dtuser))
print(f"MODFLOW time steps per day: {mf_couple_nstp }")

MODFLOW coupling frequency 1 hours
MODFLOW coupled to D-FLOW FM every 12 output time step (300.0 sec.)
MODFLOW coupling tag: 01.00H
MODFLOW time steps per day: 24


# Set a few variables for controlling coupling

In [8]:
HDRY = -1e30
DEPTH_MIN = 0.1

In [9]:
str(libmf6), libmf6.is_file()

('C:\\Users\\bbayrakt\\AppData\\Local\\miniforge3\\envs\\liss\\Scripts\\libmf6.dll',
 False)

# D-FLOW to MODFLOW weights



## GHB weights

In [10]:
fpath = f"../mapping/PJ/dflow_{dflow_grid_name}_to_{mf_grid_name}_ghb.npz"
npzfile = np.load(fpath)
print(fpath)

dflow2mfghb = npzfile["dflow2mfghb"]
print(f'dflow2ghb shape: {dflow2mfghb.shape}')
ghbmask = npzfile["ghbmask"]
print(f'ghb mask shape :{ghbmask.shape}')
ghb2qext = npzfile["ghb2qext"]
print(f'ghb2qext shape: {ghb2qext.shape}')

../mapping/PJ/dflow_LIS_GPT_PJ_cells3_net_to_PJmf6_ghb.npz
dflow2ghb shape: (354, 21660)
ghb mask shape :(354,)
ghb2qext shape: (21660, 354)


## CHD weights

In [11]:
fpath = f"../mapping/PJ/dflow_{dflow_grid_name}_to_{mf_grid_name}_chd.npz"
print(fpath)
npzfile = np.load(fpath)

dflow2mfchd = npzfile["dflow2mfchd"]
print(f'dflow2chd shape: {dflow2mfchd.shape}')
chdmask = npzfile["chdmask"]
print(f'chd mask shape :{chdmask.shape}')
chd2qext = npzfile["chd2qext"]
print(f'chd2qext shape: {chd2qext.shape}')

../mapping/PJ/dflow_LIS_GPT_PJ_cells3_net_to_PJmf6_chd.npz
dflow2chd shape: (1461, 21660)
chd mask shape :(1461,)
chd2qext shape: (21660, 1461)


# Create Modflow Model

In [12]:
# Modflow Paths
mf_base_path = pl.Path("../modflow/pj_2018_adjust_CHDA1/base/").resolve()
mf_run_path = pl.Path(f"../modflow/pj_2018_adjust_CHDA1/run_{mf_tag}_withSWMM1min/").resolve()

In [13]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path, verbosity_level=verbosity())
gwf = sim.get_model()

In [14]:
sim.set_sim_path(mf_run_path)
(mf_run_path / 'outputs').mkdir(exist_ok=True)

## Change TDIS to defined time steps

In [15]:
# tdis = sim.get_package("TDIS")
# perioddata = tdis.perioddata.array

# # mf_startdate = tdis.start_date_time.array
# # mf_perioddata= tdis.perioddata.array
# # # calculate date for each stress period
# # mf_SPdates = [pd.to_datetime(mf_startdate)]
# # for perlen, nstp, tsmult in mf_perioddata:
# #     mf_SPdates.append(mf_SPdates[-1] + pd.Timedelta(days = perlen))
# # # remove the last date because it is extra
# # mf_SPdates = mf_SPdates[:-1]
# # # Compute cumulative total times at the end of each stress period
# # mf_totaltimes= tdis.perioddata.array['perlen'].cumsum()

# # mf_tdis_df = pd.DataFrame({'Date':mf_SPdates,
# #                            'SP': list(mf_perioddata),
# #                            'totim': mf_totaltimes})

# # mf_tdis_df

# perioddata["nstp"] = nstp
# print(perioddata)

# tdis.perioddata = perioddata
# print(perioddata)

## Write the new model files

In [16]:
sim.write_simulation(silent=silent())

## Create SWMM Well Package

In [17]:
from swmm_mf_connect_1 import intersect_points_grid
#intersect_points_grid()
n_junctions, junctions, mf6_cells, swmm_inverts, possible_junctions = intersect_points_grid(sim_ws=mf_run_path , 
                                                                                            swmm_pth=str(swmm_path), 
                                                                                            swmm_pts_pth='../swmm/PJ/PJ_Sewer_GIS/Juntions_4456.shp',
                                                                                              pts_name_col='Name', crs='epsg:4456')
print(n_junctions, len(possible_junctions), junctions, mf6_cells, swmm_inverts)

SWMM Junctions/Nodes


,Id,Name,geometry,invert_elev_ft_navd88
0,0,J1,POINT (2260553.636 260346.042),160.606961
1,0,J2,POINT (2260302.775 260576.372),155.938325
2,0,J3,POINT (2260125.750 260734.668),146.328745
3,0,J4,POINT (2259957.385 260891.128),130.951448
4,0,J7,POINT (2260087.817 261363.642),143.907485


SWMM Junctions with l,r,c


,Id,Name,geometry,invert_elev_ft_navd88,row,col,idom,top,bot01,bot02,...,bot12,bot13,bot14,bot15,bot16,bot17,bot18,bot19,bot20,lay
0,0,J1,POINT (2260553.636 260346.042),160.606961,87,71,1,175.37,107.72,56.27,...,-411.08,-456.62,-502.16,-547.71,-567.74,-681.24,-724.60,-854.69,-898.05,0
1,0,J2,POINT (2260302.775 260576.372),155.938325,85,70,1,149.61,99.97,52.39,...,-407.54,-453.95,-500.36,-546.77,-563.73,-674.92,-718.49,-849.19,-892.75,0
2,0,J3,POINT (2260125.750 260734.668),146.328745,85,70,1,149.61,99.97,52.39,...,-407.54,-453.95,-500.36,-546.77,-563.73,-674.92,-718.49,-849.19,-892.75,0
3,0,J4,POINT (2259957.385 260891.128),130.951448,84,69,1,138.17,93.16,48.21,...,-404.91,-452.02,-499.14,-546.25,-561.66,-671.53,-715.21,-846.25,-889.93,0
4,0,J7,POINT (2260087.817 261363.642),143.907485,82,70,1,130.35,85.60,40.84,...,-404.59,-451.56,-498.53,-545.50,-557.85,-665.99,-709.61,-840.49,-884.12,0


99 99 ['J1', 'J2', 'J3', 'J4', 'J7', 'J6', 'J5', 'J8', 'J9', 'J10', 'J11', 'J12', 'J13', 'J14', 'J15', 'J16', 'J26', 'J25', 'J24', 'J23', 'J22', 'J21', 'J20', 'J19', 'J18', 'J17', 'J27', 'J28', 'J33', 'J32', 'J31', 'J30', 'J29', 'J37', 'J36', 'J34', 'J35', 'J38', 'J40', 'J39', 'J47', 'J46', 'J45', 'J44', 'J42', 'J41', 'J43', 'J48', 'J49', 'J50', 'J52', 'J51', 'J53', 'J54', 'J55', 'J56', 'J58', 'J57', 'J59', 'J60', 'J61', 'J62', 'J63', 'J64', 'J65', 'J66', 'J67', 'J68', 'J87', 'J80', 'J83', 'J85', 'J84', 'J82', 'J81', 'J86', 'J88', 'J89', 'J90', 'J91', 'J93', 'J92', 'J94', 'J95', 'J96', 'J97', 'J99', 'J98', 'J69', 'J70', 'J71', 'J72', 'J73', 'J74', 'J75', 'J76', 'J78', 'J79', 'J77'] {'J1': (0, 87, 71), 'J2': (0, 85, 70), 'J3': (0, 85, 70), 'J4': (0, 84, 69), 'J7': (0, 82, 70), 'J6': (0, 83, 70), 'J5': (0, 84, 70), 'J8': (0, 86, 68), 'J9': (0, 84, 68), 'J10': (0, 83, 69), 'J11': (0, 82, 68), 'J12': (0, 81, 68), 'J13': (0, 81, 69), 'J14': (0, 80, 67), 'J15': (0, 79, 67), 'J16': (0, 79, 66

In [18]:
swmm_well_sp  = []
swmm_well_obs = []
for key, value in mf6_cells.items():
    # print(key)
    # print(value)
    l = value[0]
    r = value[1]
    c = value[2]
    cell = [l,r,c,0]
    swmm_well_sp.append(cell)
    obs = (key, 'WEL', (l,r,c))
    swmm_well_obs.append(obs)

In [19]:
wel = flopy.mf6.ModflowGwfwel(
    gwf,
    save_flows=True,
    boundnames=True,
    stress_period_data={0:swmm_well_sp},
    observations= {'outputs/swmm_well_obs.csv': swmm_well_obs},
    pname='SWMM')

# Define base GHB variables

In [20]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
assert ghb_data0.shape[0] == ghbmask.shape[0]

# Define base CHD variables

2 CHD packages
1. Surface CHDs (chds in bay, and perimeter chd's in bay ,ALL in the top layer) which only has one external file
2. Perimeter CHDs (perimeter chds not in the bay, at all layers), 366 external files for every day. 

In [21]:
chd_all = gwf.get_package("chd_coast")
chd_data0 = chd_all.stress_period_data.get_dataframe()[0]
# print_value(chd_data0)
assert chd_data0.shape[0] == chdmask.shape[0]
chd_all.stress_period_data.get_dataframe()[0]

,cellid_layer,cellid_row,cellid_column,head,salinity,boundname
0,0,3,19,0.330,33.9,bay
1,0,3,20,0.330,34.6,bay
2,0,3,21,0.330,34.9,bay
3,0,3,22,0.330,34.7,bay
4,0,3,23,0.330,34.7,bay
...,...,...,...,...,...,...
1456,0,22,68,0.324,34.2,surface perimeter coastal
1457,0,24,70,0.324,34.4,surface perimeter coastal
1458,0,26,72,0.324,34.4,surface perimeter coastal
1459,0,28,74,0.324,34.4,surface perimeter coastal


# Setup and initialize D-FLOW FM
You will need to set `dflow_dirpath` to the correct directory on your machine.

## Paths

In [22]:
dflow_dirpath = pl.Path(r"..\dflow-fm\dflowfm_dll").resolve()
dflow_base = pl.Path(r"..\dflow-fm\coarse\tides_2018_atm\base").resolve()
dflow_working = pl.Path(r"..\dflow-fm\coarse\tides_2018_atm\run").resolve()
dflow_config = dflow_working / "FlowFM.mdu"
print(dflow_config)

D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\dflow-fm\coarse\tides_2018_atm\run\FlowFM.mdu


In [23]:
if dflow_working.is_dir():
    shutil.rmtree(dflow_working)
shutil.copytree(dflow_base, dflow_working)
(dflow_working / "output").mkdir(parents=True, exist_ok=True)

In [24]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(dflow_dirpath) + os.pathsep + os.environ["PATH"]
)

In [25]:
(pl.Path(dflow_dirpath) / "dflowfm.dll").is_file()

True

## Initialize D-Flow FM API

In [26]:
dflowfm = BMIWrapper(
    engine="dflowfm",
    configfile=str(dflow_config),
)

In [27]:
import os
print("cwd:", os.getcwd())

cwd: d:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\notebooks-PJ


In [28]:
dflowfm.initialize() # this line of code is what changes the cwd to  '\dflow-fm\coarse\tides\run'

In [29]:
import os
print("cwd:", os.getcwd())

cwd: D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\dflow-fm\coarse\tides_2018_atm\run


## Get data using DFLOW API

In [30]:
ndxi = int(dflowfm.get_var("ndxi"))
ndx = int(dflowfm.get_var("ndx")) # number of nodes
x = dflowfm.get_var("xz")
y = dflowfm.get_var("yz")
z = dflowfm.get_var("bl")
xy = [(xx, yy) for (xx, yy) in zip(x, y)]

ndx, ndxi, x.shape, y.shape # bnb note: are these values ok? answer: they shouldnt matter because they run with the gp model 

(21755, 21660, (21755,), (21755,))

In [31]:
def read_mdu(path):
    config = {}
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):  # skip comments
                continue
            if "=" in line:
                key, val = [s.strip() for s in line.split("=", 1)]
                config[key.lower()] = val
    return config

mdu_path = r"D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\dflow-fm\coarse\tides_2018_atm\run\FlowFM.mdu"
mdu_data = read_mdu(mdu_path)
dflow_start_date = mdu_data["refdate"].split()[0]
dflow_start_date = datetime.datetime.strptime(dflow_start_date, "%Y%m%d")
dflow_end_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_end_time())
print(f'DFlow start date: {dflow_start_date} \nDFlow end date: {dflow_end_date}')

DFlow start date: 2018-09-20 00:00:00 
DFlow end date: 2018-09-30 00:00:00


In [32]:
qext = np.zeros(ndx)
qext.shape, qext

((21755,), array([0., 0., 0., ..., 0., 0., 0.]))

In [33]:
qext_cum = np.zeros(ndx)
qext_cum.shape

(21755,)

In [34]:
vextcum = dflowfm.get_var("vextcum")
vextcum.shape, vextcum

((21755,), array([0., 0., 0., ..., 0., 0., 0.]))

# Initialize SWMM

In [35]:
with pyswmm.Simulation(str(swmm_path)) as swmm_sim:
    swmm_start_date = swmm_sim.start_time
    swmm_end_date = swmm_sim.end_time
    print(swmm_start_date)
    print(swmm_end_date)

swmm_sim = pyswmm.Simulation(str(swmm_path))
swmm_nodes = {}
for j in junctions:
    swmm_nodes[j] = pyswmm.Nodes(swmm_sim)[j]

2018-09-20 23:49:00
2018-10-02 00:00:00


In [36]:
swmm_sim.start()

# Initialize MODFLOW using MODFLOW API

## Change Modflow TDIS

In [37]:
tdis = sim.get_package("TDIS")
mf_perioddata= tdis.perioddata.array
mf6_start_date = datetime.datetime.strptime(tdis.start_date_time.get_data(), "%Y-%m-%d")
# calculate date for each stress period
mf_SPdates = [mf6_start_date]
for perlen, _, _ in mf_perioddata:
    mf_SPdates.append(mf_SPdates[-1] + pd.Timedelta(days = perlen))
# remove the last date because it is extra
mf_SPdates = mf_SPdates[:-1]

# Compute cumulative total times at the end of each stress period
mf_totaltimes= tdis.perioddata.array['perlen'].cumsum()

mf_tdis_df = pd.DataFrame({'Date':mf_SPdates,
                           'SP_data': list(mf_perioddata),
                           'SP': [i for i in range(len(mf_perioddata))],
                           'totim': mf_totaltimes})

In [38]:

# latest date among models
latest_date = max(swmm_start_date, dflow_start_date, mf6_start_date)
print("Latest date:", latest_date)

# find closest available date in mf_tdis_df
closest_date_idx = (mf_tdis_df['Date'] - latest_date).abs().idxmin() 
print(closest_date_idx)
closest_date = mf_tdis_df.loc[closest_date_idx, 'Date']

# get corresponding stress period
latest_date_SP = mf_tdis_df.loc[closest_date_idx, 'SP']

print("Closest available date:", closest_date)
print("Associated SP:", latest_date_SP)

print(f'Modflow Stress Period associated with the lastest models start date: {latest_date_SP}')
assert latest_date_SP + len(mf_perioddata[latest_date_SP:]) == len(mf_tdis_df)
# Update nstp where SP index >= latest_date_SP
mf_perioddata["nstp"][latest_date_SP:] = mf_couple_nstp 
mf_perioddata

mf_tdis_df['SP_data_coupled'] = list(mf_perioddata)


# CHANGE TDIS IN MF MODEL TO REFLECT TIME STEP CHANGES
tdis.mf_perioddata = mf_perioddata
print(mf_perioddata)


Latest date: 2018-09-20 23:49:00
264
Closest available date: 2018-09-21 00:00:00
Associated SP: 264
Modflow Stress Period associated with the lastest models start date: 264
[(365.,  3, 1.3) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. )
 (  1.,  1, 1. ) (  1.,  1, 1. ) (  1.,  1, 1. 

In [39]:
sim.write_simulation(silent=silent())


## MF API

In [40]:
# set the mdll_path to be the absolute path where the mf6 dll is located
#mdll_path = pl.Path(r"D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\mf6dll\libmf6.dll")
mdll_path = str(r"D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\mf6dll\libmf6.dll")
print(mdll_path)

mf_run_path = str(mf_run_path)
print(mf_run_path)

D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\mf6dll\libmf6.dll
D:\LISS_GW\GitRepo_CoupledModels\nywsc_compound_flooding\modflow\pj_2018_adjust_CHDA1\run_01.00H_withSWMM1min


In [41]:
mf6 = ModflowApi(mdll_path, working_directory=mf_run_path)

In [42]:
mf6.initialize()

## use recent modflowapi functionality for accessing data from the api in flopy-like data structures

In [43]:
apisim = ApiSimulation.load(mf6)
apiml = apisim.get_model()

sewer_flow = apiml.get_package("swmm")
swmm_dtype = [("nodelist", "O"), ("q", float)]

## Define MODFLOW variable tags and set pointer to MODFLOW variables

In [44]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "GWF", "GHB")
ghb_cond_tag = mf6.get_var_address("COND", "GWF", "GHB")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "GHB")

In [45]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

In [46]:
chd_head_tag = mf6.get_var_address("HEAD", "GWF", "CHD_coast")
chd_flow_tag = mf6.get_var_address("SIMVALS", "GWF", "CHD_coast")

In [47]:
chd_head_ptr = mf6.get_value_ptr(chd_head_tag)
chd_flow = np.zeros(chd_head_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [48]:
ghb_elev_dict = {}
ghb_cond_dict = {}
chd_elev_dict = {}
qext_dict = {}
swmm_q_dict = {}

#### Function to update MODFLOW GHB data

find which ghb+chd cells have a botm01 elevation lower 0 
    - exclude cells from coupling? 
    or 
    -raise heads to be above those cells

In [49]:
def update_mf(key, s, d):
    #print('running update_mf')
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0

    ghb_head = ghb_data0["bhead"].to_numpy()
    # print(f'ghb_data0 : {ghb_data0["bhead"].shape}') 
    # print(f'ghb_head :{ghb_head.shape}')

    ghb_head[ghbmask] = dflow2mfghb.dot(s)[ghbmask] * m2ft
    # print(f'ghbmask : {ghbmask.shape} , dflow2mfghb :{dflow2mfghb.shape}')

    ghb_cond = ghb_data0["cond"].to_numpy()
    # print(f'ghb_cond : {ghb_data0["cond"].shape}') 

    ghb_cond[ghbmask] = ghb_cond[ghbmask] * dflow2mfghb.dot(mult)[ghbmask]
    
    ghb_bhead_ptr[:] = ghb_head[:] # does not work
    #ghb_bhead_ptr[:] = ghb_bhead_ptr[:] * 1.5 # _ptr variable is a pointer for the APR. 

    ghb_cond_ptr[:] = ghb_cond[:] #does not work
    #ghb_cond_ptr[:] = ghb_cond_ptr[:]
    
    chd_head = chd_data0["head"].to_numpy()

    chd_head[chdmask] = dflow2mfchd.dot(s)[chdmask] * m2ft
    
    chd_head_ptr[:] = chd_head[:] # does not work
    #chd_head_ptr[:] = chd_head_ptr[:] * 1.5 #this works
   
    # update results dictionary
    ghb_elev_dict[key] = ghb_head.copy()
    ghb_cond_dict[key] = ghb_cond.copy()
    chd_elev_dict[key] = chd_head.copy()

#### Function to update D-Flow FM Qext data

In [50]:
def update_dflow(key, d):
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    #print(f'ghb_flow: {ghb_flow.shape, mf6.get_value(ghb_flow_tag).shape}')
    
    dflow_qext_ghb = ghb2qext.dot(ghb_flow)
    #print(f'dflow_qext_ghb{dflow_qext_ghb.shape, ghb2qext.shape, ghb_flow.shape}')

    dflow_qext_ghb[d == 0.0] = 0.0
    
    chd_flow = -mf6.get_value(chd_flow_tag) * cfd2cms
    #print(f'chd_flow{chd_flow.shape, mf6.get_value(chd_flow_tag).shape}')

    dflow_qext_chd = chd2qext.dot(chd_flow)

    dflow_qext_chd[d == 0.0] = 0.0

    dflow_qext = dflow_qext_ghb + dflow_qext_chd
    
    qext_cum[:ndxi] += dflow_qext[:ndxi]
    qext[:ndxi] = dflow_qext[:ndxi]
    dflowfm.set_var("qext", qext)

    # update results dictionaries
    qext_dict[key] = qext[:ndxi].copy()

#### Function to update SWMM and the MODFLOW SWMM well

In [51]:
def update_swmm(key):
    heads = apiml.X
    mf6_spd = []
    for key, value in swmm_nodes.items():
        _, row, col = mf6_cells[key]
        # using layer 0 should be ok even if the junction is not in that layer (?)
        # consider changing this to grab head from the correct layer where the junction is, but it's ok to keep for now
        head = heads[0, row, col]
        # might be nice to use the simulated swmm water-level
        # in the junctions instead of the invert elevation
        pot = head - swmm_inverts[key] * m2ft
        if pot > 0.0:
            Q = pot * 0.001 # this coefficient may be changed
        else:
            Q = pot * 0.0002
        value.generated_inflow(Q * cfd2cms)  # convert to CMS
        mf6_spd.append(((0, row, col), -Q)) # unit conversion needed

    # update gwf_swmm well file with new flux data - will be used next time step
    mf6_spd = np.array(mf6_spd, dtype=swmm_dtype)
    sewer_flow.stress_period_data.values = mf6_spd

    # update results dictionary
    swmm_q_dict[key] = mf6_spd["q"].copy()


#### Run each time step

In [52]:
print(
    f"DFLOWFM current_time: {dflowfm.get_current_time():15,.1f} sec. ({dflowfm.get_current_time()/86400.:15,.1f} days)\n"
     + f"DFLOWFM end_time:     {dflowfm.get_end_time():15,.1f} sec. ({dflowfm.get_end_time()/86400.:15,.1f} days)"
)

DFLOWFM current_time:             0.0 sec. (            0.0 days)
DFLOWFM end_time:           864,000.0 sec. (           10.0 days)


In [53]:
import datetime
mf6_start_date = datetime.datetime.strptime(tdis.start_date_time.get_data(), "%Y-%m-%d")
mf6_end_date = mf6_start_date + datetime.timedelta(days=mf6.get_end_time())
print(f'MF start date: {mf6_start_date} \nMF end date: {mf6_end_date}')

MF start date: 2017-01-01 00:00:00 
MF end date: 2019-01-01 00:00:00


In [54]:
print(f'MF start date: {mf6_start_date}')
print(f'SWMM start date: {swmm_start_date}')
print(f'Dflow start date: {dflow_start_date}')
print('-------------------------------------')

idx = 0
jdx = 0
t0 = time.perf_counter()

# NOTE: This workflow assume that the modflow model will run the longest, and that modflow has the earliest start date
start_date = mf6_start_date
current_date = mf6_start_date

print("Initializing Modflow Sim")
#==============================================================
# STEP 1 : Run MF until one of the other two models is "ready"
#=============================================================
while (current_date < dflow_start_date) & (current_date < swmm_start_date):
    mf6.update()
    current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
    #print(current_date)
print(f'MF current date at the end of step 1 : {current_date} \n  Dflow start date is {dflow_start_date}')
print('---------------------------------')

#====================================
# IF DFLOW START DATE > SWMM START DATE
#======================================
# STEP 2: 
#===================
if (current_date >= dflow_start_date) & (current_date < swmm_start_date):
    print(f'Step 2: Running MF and DFLOW (separately), waiting for SWMM')
    #current_date_df = dflow_start_date # start tracking dflow time
    while (current_date < swmm_start_date):
        mf6.update()
        current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
        dflowfm.update() 
        current_date_df = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())
        print(f'mf:{current_date} , dflow: {current_date_df}',end="\r")

    # Continue updating dflow if its time is not lined up with MF
    while current_date_df<current_date:# Make sure by the end of step two, dflow and modflow current times are the same
        dflowfm.update() 
        current_date_df = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())
        print(f'mf:{current_date} , dflow: {current_date_df}',end="\r")
    assert current_date >= swmm_start_date
    # swmm_time = swmm_sim.current_time
    # # If MF time advances past swmm time
    # while swmm_time < current_date:
    #     dt_sec = mf6.get_time_step() * d2sec
    #     swmm_sim.step_advance(int(dt_sec))
    #     swmm_time = swmm_sim.current_time

    print(f'End of Step 2: MF date: {current_date} , DFlow date: {current_date_df}, SWMM date:{swmm_sim.current_time}')
    print('---------------------------------')
        
    # STEP 3: Cpuple all three models 
    print('Coupling all 3 models...')
    while current_date <= dflow_end_date:
        idx += 1
        frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days
        print(f"(Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
        dflowfm.update()
        current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

        if idx == int(dflow_per_mf):
            print(f"(*Coupling* - Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
            print(' ')
            print(f'dflow: {dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())}, swmm: {swmm_sim.current_time}, mf: {start_date + datetime.timedelta(days=mf6.get_current_time())}')
            s = dflowfm.get_var("s1")[:ndxi] # water level
            d = dflowfm.get_var("hs")[:ndxi]
            
            mf6.prepare_time_step(mf6.get_time_step())
            update_mf(str(jdx), s, d)
            mf6.do_time_step()
            mf6.finalize_time_step()
            update_dflow(str(jdx), d)
            
            # advance SWMM
            update_swmm(str(jdx))
            dt_sec = mf6.get_time_step() * d2sec
            swmm_sim.step_advance(int(dt_sec))
            try:
                swmm_sim.__next__()  
            except StopIteration:
                break        

            # update counters
            idx = 0
            jdx += 1
        
        if current_date >= dflow_end_date:
            break
#====================================
# IF SWMM START DATE > DFLOW START DATE
#======================================
# STEP 2: 
#===================
if (current_date >= swmm_start_date) & (current_date < dflow_start_date):
    print(f'Step 2: Running MF and SWMM (separately), waiting for Dflow')
    while current_date < dflow_start_date:
        mf6.update()
        dt_sec = mf6.get_time_step() * d2sec
        swmm_sim.step_advance(int(dt_sec)) 
        current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
        print(f'mf:{current_date} , swmm:{swmm_sim.current_time}')

    print(f'End of Step 2: MF date:{current_date}, SWMM date: {swmm_sim.current_time}')
    print('---------------------------------')
    
    # STEP 3: Cpuple all three models 
    print('Coupling all 3 models...')
    while current_date <= dflow_end_date:
        idx += 1
        frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days
        print(f"(Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
        dflowfm.update()
        current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

        if idx == int(dflow_per_mf):
            print(f"(*Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
            s = dflowfm.get_var("s1")[:ndxi] # water level
            d = dflowfm.get_var("hs")[:ndxi]
            
            mf6.prepare_time_step(mf6.get_time_step())
            update_mf(str(jdx), s, d)
            mf6.do_time_step()
            mf6.finalize_time_step()
            update_dflow(str(jdx), d)
            
            # advance SWMM
            update_swmm(str(jdx))
            dt_sec = mf6.get_time_step() * d2sec
            swmm_sim.step_advance(int(dt_sec))

            try:
                swmm_sim.__next__()  
            except StopIteration:
                break        

            # update counters
            idx = 0
            jdx += 1
            
        
        if current_date >= dflow_end_date:
            break
#====================================
# IF SWMM START DATE == DFLOW START DATE
#======================================
# STEP 2: 
#===================
if (current_date == dflow_start_date) & (current_date == swmm_start_date):
    print('Running MF + Dflow + SWMM')
    print(f'current date: {mf6_start_date}')
    print(f'SWMM start date: {swmm_start_date}')
    print(f'Dflow start date: {dflow_start_date}')
    print('-------------------------------------')
    # STEP 3: Cpuple all three models 
    while current_date <= dflow_end_date:
        idx += 1
        frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days
        print(f"(Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
        dflowfm.update()
        current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

        if idx == int(dflow_per_mf):
            print(f"(*Coupling* - Current date: {current_date}) - {frac_comp:6.2%} complete - ({idx:03d})    ", end="\r")
            s = dflowfm.get_var("s1")[:ndxi] # water level
            d = dflowfm.get_var("hs")[:ndxi]
            
            mf6.prepare_time_step(mf6.get_time_step())
            update_mf(str(jdx), s, d)
            mf6.do_time_step()
            mf6.finalize_time_step()
            update_dflow(str(jdx), d)
            
            # advance SWMM
            update_swmm(str(jdx))
            dt_sec = mf6.get_time_step() * d2sec
            swmm_sim.step_advance(int(dt_sec))
            try:
                swmm_sim.__next__()  
            except StopIteration:
                break        

            # update counters
            idx = 0
            jdx += 1
        
        if current_date >= dflow_end_date:
            break

MF start date: 2017-01-01 00:00:00
SWMM start date: 2018-09-20 23:49:00
Dflow start date: 2018-09-20 00:00:00
-------------------------------------
Initializing Modflow Sim
MF current date at the end of step 1 : 2018-09-20 00:00:00 
  Dflow start date is 2018-09-20 00:00:00
---------------------------------
Step 2: Running MF and DFLOW (separately), waiting for SWMM
End of Step 2: MF date: 2018-09-21 00:00:00 , DFlow date: 2018-09-21 00:00:00, SWMM date:2018-09-20 23:49:00
---------------------------------
Coupling all 3 models...
 *Coupling* - Current date: 2018-09-21 01:00:00) - 98.74% complete - (012)    
dflow: 2018-09-21 01:00:00, swmm: 2018-09-20 23:49:00, mf: 2018-09-21 00:00:00
 *Coupling* - Current date: 2018-09-21 02:00:00) - 98.74% complete - (012)    
dflow: 2018-09-21 02:00:00, swmm: 2018-09-21 00:49:00, mf: 2018-09-21 01:00:00
 *Coupling* - Current date: 2018-09-21 03:00:00) - 98.74% complete - (012)    
dflow: 2018-09-21 03:00:00, swmm: 2018-09-21 01:49:00, mf: 2018-09-2

In [55]:
# print(f"MF start date: {mf6_start_date}")
# print(f"SWMM start date: {swmm_start_date}")
# print(f"Dflow start date: {dflow_start_date}")
# print('-------------------------------------')

# print("Initializing Modflow Sim")

# idx = 0
# jdx = 0
# t0 = time.perf_counter()

# start_date = mf6_start_date
# current_date = mf6_start_date

# # ==============================================================
# # STEP 1 : Run MF until one of the other two models is "ready"
# # ==============================================================
# while (current_date < dflow_start_date) & (current_date < swmm_start_date):
#     mf6.update()
#     current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
#     print(f"MF current date: {current_date}", end="\r", flush=True)
# print()  # newline after loop

# print(f"Dates at the end of step 1 : {current_date} | Dflow start date: {dflow_start_date}")
# print('---------------------------------')

# # =============================================================
# # STEP 2: DFLOW starts before SWMM
# # =============================================================
# if (current_date >= dflow_start_date) & (current_date < swmm_start_date):
#     print("Running MF and DFLOW (separately), waiting for SWMM")

#     while current_date < swmm_start_date:
#         mf6.update()
#         dflowfm.update()
#         current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
#         current_date1 = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())
#         print(f"MF: {current_date} | DFLOW: {current_date1}", end="\r", flush=True)
#     print()  # newline after loop

#     print(f"End of Step 2: MF {current_date} | DFLOW {current_date1}")
#     print('---------------------------------')

#     # =============================================================
#     # STEP 3: Couple all three models
#     # =============================================================
#     while current_date <= dflow_end_date:
#         idx += 1
#         frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days

#         # Progress updates (overwrite themselves)
#         print(f"Current date: {current_date} | Fraction complete: {frac_comp:6.2%} | Coupling idx: {idx:03d}", end="\r", flush=True)
#         print(f"SWMM time: {swmm_sim.current_time}", end="\r", flush=True)

#         dflowfm.update()
#         current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

#         if idx == int(dflow_per_mf):
#             # Coupling step executed (overwrite same line)
#             print(f"*Coupling executed* @ {current_date} | Fraction complete: {frac_comp:6.2%}", end="\r", flush=True)

#             # Update models
#             s = dflowfm.get_var("s1")[:ndxi]
#             d = dflowfm.get_var("hs")[:ndxi]

#             mf6.prepare_time_step(mf6.get_time_step())
#             update_mf(str(jdx), s, d)
#             mf6.do_time_step()
#             mf6.finalize_time_step()
#             update_dflow(str(jdx), d)

#             update_swmm(str(jdx))
#             dt_sec = mf6.get_time_step() * d2sec
#             swmm_sim.step_advance(int(dt_sec))

#             try:
#                 swmm_sim.__next__()
#             except StopIteration:
#                 break

#             idx = 0
#             jdx += 1

#         if current_date >= dflow_end_date:
#             break
#     print()  # newline after Step 3

# # =============================================================
# # STEP 2: SWMM starts before DFLOW
# # =============================================================
# if (current_date >= swmm_start_date) & (current_date < dflow_start_date):
#     print("Running MF and SWMM (separately), waiting for Dflow")

#     while current_date < dflow_start_date:
#         mf6.update()
#         dt_sec = mf6.get_time_step() * d2sec
#         swmm_sim.step_advance(int(dt_sec))
#         current_date = start_date + datetime.timedelta(days=mf6.get_current_time())
#         print(f"MF: {current_date} | SWMM: {swmm_sim.current_time}", end="\r", flush=True)
#     print()  # newline after loop

#     print(f"End of Step 2: MF {current_date} | SWMM {swmm_sim.current_time}")
#     print('---------------------------------')

#     while current_date <= dflow_end_date:
#         idx += 1
#         frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days
#         print(f"Current date: {current_date} | Fraction complete: {frac_comp:6.2%} | Coupling idx: {idx:03d}", end="\r", flush=True)

#         dflowfm.update()
#         current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

#         if idx == int(dflow_per_mf):
#             print(f"*Coupling executed* @ {current_date} | Fraction complete: {frac_comp:6.2%}", end="\r", flush=True)

#             s = dflowfm.get_var("s1")[:ndxi]
#             d = dflowfm.get_var("hs")[:ndxi]

#             mf6.prepare_time_step(mf6.get_time_step())
#             update_mf(str(jdx), s, d)
#             mf6.do_time_step()
#             mf6.finalize_time_step()
#             update_dflow(str(jdx), d)

#             update_swmm(str(jdx))
#             dt_sec = mf6.get_time_step() * d2sec
#             swmm_sim.step_advance(int(dt_sec))

#             try:
#                 swmm_sim.__next__()
#             except StopIteration:
#                 break

#             idx = 0
#             jdx += 1

#         if current_date >= dflow_end_date:
#             break
#     print()  # newline after Step 3

# # =============================================================
# # STEP 2: SWMM and DFLOW start same time
# # =============================================================
# if (current_date == dflow_start_date) & (current_date == swmm_start_date):
#     print(f"Running MF + DFLOW + SWMM | Start date: {mf6_start_date}")

#     while current_date <= dflow_end_date:
#         idx += 1
#         frac_comp = 1 - (dflow_end_date - current_date).days / (dflow_end_date - start_date).days
#         print(f"Current date: {current_date} | Fraction complete: {frac_comp:6.2%} | Coupling idx: {idx:03d}", end="\r", flush=True)

#         dflowfm.update()
#         current_date = dflow_start_date + datetime.timedelta(seconds=dflowfm.get_current_time())

#         if idx == int(dflow_per_mf):
#             print(f"*Coupling executed* @ {current_date} | Fraction complete: {frac_comp:6.2%}", end="\r", flush=True)

#             s = dflowfm.get_var("s1")[:ndxi]
#             d = dflowfm.get_var("hs")[:ndxi]

#             mf6.prepare_time_step(mf6.get_time_step())
#             update_mf(str(jdx), s, d)
#             mf6.do_time_step()
#             mf6.finalize_time_step()
#             update_dflow(str(jdx), d)

#             update_swmm(str(jdx))
#             dt_sec = mf6.get_time_step() * d2sec
#             swmm_sim.step_advance(int(dt_sec))

#             try:
#                 swmm_sim.__next__()
#             except StopIteration:
#                 break

#             idx = 0
#             jdx += 1

#         if current_date >= dflow_end_date:
#             break
#     print()  # newline at end


#### Finalize models

In [56]:
mf6.finalize()

In [57]:
# swmm_sim.terminate_simulation()
# swmm_sim.report()
# swmm_sim.close()

In [58]:
dflowfm.finalize()

#### Save ghb elevation and conductance data to compressed files

In [59]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **ghb_elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **ghb_cond_dict)

#### Save chd elevation to compressed file

In [60]:
np.savez_compressed(f"{mf_run_path}/chd_elev.npz", **chd_elev_dict)

#### Save qext data to compressed file

In [61]:
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)

#### Save SWMM flux data to compressed file

In [62]:
# np.savez_compressed(f"{mf_run_path}/swmm_q.npz", **swmm_q_dict)